Import Statements

In [ ]:
pip install --upgrade gensim

In [ ]:
import pandas as pd
import numpy as np
import csv
import json
from itertools import islice
from collections import OrderedDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from glob import glob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tqdm import tqdm
import pickle
import math
import re
from sklearn.model_selection import train_test_split
import operator
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats
! pip install git+https://github.com/LIAAD/yake
import yake
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index
  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-utbygd18
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-utbygd18


In [ ]:
import logging
import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex

In [ ]:
# import gensim.downloader as api
# model = api.load('word2vec-google-news-300')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file1 = open("/content/drive/MyDrive/Tweelink_Dataset/twitter_base_preprocessed.pkl", "rb")
df = pickle.load(file1)
file1.close()

In [ ]:
df.head()

,author id,created_at,geo,country,country_code,place_full_name,place_name,place_type,id,lang,...,source,tweet,hashtags,sensitive,urls,context_text,context_probability,context_type,Preprocessed_Data,Date_Only
0,1952510090.0,2022-02-14 00:11:44+00:00,,,,,,,1493015060943454208.0,en,...,Twitter for Android,ð¥ð¥ð¥ð¥ð¥ð¥ð¥â¤µï¸â¤µï¸â¤µï¸...,"CommonSenseGunLaws,GunControlNow,GunSafes,GunS...",False,https://t.co/LjkEup24Dk,,0.0,,[ð¥ð¥ð¥ð¥ð¥ð¥ð¥â¤µï¸â¤µï¸â¤µï¸...,2022-02-14
1,20453105.0,2022-02-14 00:04:56+00:00,,,,,,,1493013352938934272.0,en,...,Twitter for Android,"It's too bad these guys are afraid of needles,...","ClownConvoy,FreeDumbConvoy,OttawaOccupied",False,https://t.co/MmFzuFjIDR,,0.0,,"[bad, guy, afraid, needle, twinrix, would, pre...",2022-02-14
2,1492660754998247424.0,2022-02-14 00:39:55+00:00,,,,,,,1493022155239534336.0,en,...,Twitter for Android,Cowboy dressed as #Furries now available at th...,"Furries,RamRanch,OttawaOccupied",False,https://t.co/GBuBCUtpXe,,0.0,,"[cowboy, dressed, furries, available, ramranch...",2022-02-14
3,117812637.0,2022-02-14 00:20:05+00:00,,,,,,,1493017163485044736.0,en,...,Twitter for iPhone,We blocked these trucks from entering the dntn...,"Riverside,OttawaOccupied",False,https://t.co/KzLkBZvjgD,,0.0,,"[blocked, truck, entering, dntn, core, riversi...",2022-02-14
4,1490044524604928000.0,2022-02-14 00:10:26+00:00,,,,,,,1493014734903382016.0,en,...,Twitter for iPhone,Krista is very pleased with how the @RCMPONT r...,"FluTruxKlanGoHome,OttawaOccupied,kkkonvoy,Otta...",False,https://t.co/kAm5KNugdA,,0.0,,"[krista, pleased, rcmpont, responded, maskless...",2022-02-14


In [ ]:
u_base_hashtag = input("Enter base hashtag: ")
u_time = input("Enter time: ")
u_location = input("Enter Location: ")

Enter base hashtag: hijab
Enter time: 2022-02-19
Enter Location: India


In [ ]:
import datetime
tweet_query = []
format = '%Y-%m-%d'
u_present_date = datetime.datetime.strptime(u_time, format)
u_prev_date = u_present_date - datetime.timedelta(days=1)
u_next_date = u_present_date + datetime.timedelta(days=1)
df_query = df.loc[df['hashtags'].str.contains(u_base_hashtag) & df['Date_Only'].isin([str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())])]

In [ ]:
df_query.head()

,author id,created_at,geo,country,country_code,place_full_name,place_name,place_type,id,lang,...,source,tweet,hashtags,sensitive,urls,context_text,context_probability,context_type,Preprocessed_Data,Date_Only
12546,123851279.0,2022-02-19 12:59:42+00:00,,,,,,,1495020266862174208.0,en,...,Twitter for iPhone,They are so desperate they have let loose #Yat...,"YatiNarsinghanand,PlotToKillModi,hijab",False,,,0.0,,"[desperate, let, loose, yatinarsinghanand, vio...",2022-02-19
8192,3187630801,2022-02-20 00:55:03+00:00,,,,,,,1495200290210910209,en,...,Twitter for Android,No Woman is Forced to wear and Not to wear Som...,"hijab,HijabIsOurPride,HijabIsFundamentalRight,...",False,,,0.0,,"[woman, forced, wear, wear, something, hijab, ...",2022-02-20
8198,2443976946,2022-02-20 00:14:25+00:00,,,,,,,1495190066112651265,en,...,Twitter Web App,Protests in US cities against #Karnataka #hija...,"Karnataka,hijab,Islamophobic,BJP",False,"https://t.co/957kBM8AJT,https://t.co/5gY741DZP...",,0.0,,"[protest, u, city, karnataka, hijab, ban, indi...",2022-02-20
8278,1483569403425959939,2022-02-20 01:06:11+00:00,18810aa5b43e76c7,Verenigde Staten,US,"Dallas, TX",Dallas,city,1495203094224904192,en,...,Twitter for iPhone,Protest by local Dallas Muslims at JFK square ...,hijab,False,https://t.co/KCm413Yb8N,,0.0,,"[protest, local, dallas, muslim, jfk, square, ...",2022-02-20
8280,18931596,2022-02-20 01:55:42+00:00,,,,,,,1495215554508075011,en,...,Twitter Web App,Feel sorry for #zahirawasim. Religion must ma...,"zahirawasim,hijab",False,,,0.0,,"[feel, sorry, zahirawasim, religion, must, mak...",2022-02-20


In [ ]:
df_query

,author id,created_at,geo,country,country_code,place_full_name,place_name,place_type,id,lang,...,source,tweet,hashtags,sensitive,urls,context_text,context_probability,context_type,Preprocessed_Data,Date_Only
12546,123851279.0,2022-02-19 12:59:42+00:00,,,,,,,1495020266862174208.0,en,...,Twitter for iPhone,They are so desperate they have let loose #Yat...,"YatiNarsinghanand,PlotToKillModi,hijab",False,,,0.0,,"[desperate, let, loose, yatinarsinghanand, vio...",2022-02-19
8192,3187630801,2022-02-20 00:55:03+00:00,,,,,,,1495200290210910209,en,...,Twitter for Android,No Woman is Forced to wear and Not to wear Som...,"hijab,HijabIsOurPride,HijabIsFundamentalRight,...",False,,,0.0,,"[woman, forced, wear, wear, something, hijab, ...",2022-02-20
8198,2443976946,2022-02-20 00:14:25+00:00,,,,,,,1495190066112651265,en,...,Twitter Web App,Protests in US cities against #Karnataka #hija...,"Karnataka,hijab,Islamophobic,BJP",False,"https://t.co/957kBM8AJT,https://t.co/5gY741DZP...",,0.0,,"[protest, u, city, karnataka, hijab, ban, indi...",2022-02-20
8278,1483569403425959939,2022-02-20 01:06:11+00:00,18810aa5b43e76c7,Verenigde Staten,US,"Dallas, TX",Dallas,city,1495203094224904192,en,...,Twitter for iPhone,Protest by local Dallas Muslims at JFK square ...,hijab,False,https://t.co/KCm413Yb8N,,0.0,,"[protest, local, dallas, muslim, jfk, square, ...",2022-02-20
8280,18931596,2022-02-20 01:55:42+00:00,,,,,,,1495215554508075011,en,...,Twitter Web App,Feel sorry for #zahirawasim. Religion must ma...,"zahirawasim,hijab",False,,,0.0,,"[feel, sorry, zahirawasim, religion, must, mak...",2022-02-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11300,3022076697,2022-02-20 23:02:11+00:00,,,,,,,1495534273063006214,en,...,Post Studio,Looking for an Modest dress for your upcoming ...,"modestfashion,hijab,abaya,muslim",False,"https://t.co/8p49EqGJ5z,https://t.co/dCiENpIuaX",,0.0,,"[looking, modest, dress, upcoming, event, ð...",2022-02-20
11306,1407681110511964160,2022-02-20 23:36:07+00:00,,,,,,,1495542813240565761,en,...,Twitter for iPhone,"Weak Hindus worship Satan, do not have manline...","India,hijab,Hindus",False,,,0.0,,"[weak, hindu, worship, satan, manliness, coura...",2022-02-20
18929,1463168868063399936.0,2022-02-18 06:39:52+00:00,,,,,,,1494562291458756608.0,en,...,Twitter for iPhone,ðð¿ð²ð®ð ð±ð²ð°ð¶ðð¶ð...,"hijab,hijabday,PopularFrontDay,Muslim,Markhor,...",False,,,0.0,,"[ðð¿ð²ð®ð, ð±ð²ð°ð¶ðð...",2022-02-18
23847,158374641.0,2022-02-19 13:46:22+00:00,,,,,,,1495032011148660736.0,en,...,Twitter Web App,@RajeevRai https://t.co/vIHQ6mIJOJ #abuazmi co...,"abuazmi,Ahmedabadblast2008,AkhileshYadav,Samaj...",False,https://t.co/vIHQ6mIJOJ,,0.0,,"[rajeevrai, http, tcovihq6mijoj, abuazmi, conf...",2022-02-19


In [ ]:
def keyword_extractor(dataset):
  preprocessed_vocabulary = dict()

  #Converting to lowercase
  def to_lower_case(text):
    text = text.lower()
    return text

  def remove_at_word(text):
    data = text.split()
    data = [d for d in data if d[0]!='@']
    text = ' '.join(data)
    return text

  def remove_hashtag(text):
    data = text.split()
    data = [d if (d[0]!='#' or len(d) == 1) else d[1:] for d in data]
    data = [d for d in data if d[0]!='#']
    text = ' '.join(data)
    return text

  def remove_URL(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'bit.ly\S+', '', text, flags=re.MULTILINE)
    return text

  #Removing stopwords
  def remove_stopwords(text):
    stopword = stopwords.words('english')
    new_list = [x for x in text.split() if x not in stopword]
    return ' '.join(new_list)

  #Removing punctuations
  def remove_punctuations(text):
    punctuations = '''!()-[|]`{};:'"\,<>./?@#$=+%^&*_~'''
    new_list = ['' if x in punctuations else x for x in text.split()]
    new_list_final = []
    for token in new_list:
      new_token=""
      for char in token:
        if(char not in punctuations):
          new_token+=char
      if(len(new_token)!=0):
        new_list_final.append(new_token)
    return ' '.join(new_list_final)

  #Tokenization
  def tokenization(text):
    return word_tokenize(text)

  def pre_process(text):
    text = to_lower_case(text)
    text = remove_at_word(text)
    text = remove_hashtag(text)
    text = remove_URL(text)
    text = remove_stopwords(text)
    text = remove_punctuations(text)
    text = tokenization(text)
    for token in text:
      if token in preprocessed_vocabulary.keys():
        preprocessed_vocabulary[token] += 1
      else:
        preprocessed_vocabulary[token] = 1
    return text
  
  preprocessed_data = [pre_process(text) for text in dataset]

  #print(preprocessed_vocabulary)

  AOF_coefficient = sum(preprocessed_vocabulary.values())/len(preprocessed_vocabulary)
  vocabulary = {token.strip():preprocessed_vocabulary[token] for token in preprocessed_vocabulary.keys() if preprocessed_vocabulary[token] > AOF_coefficient and len(token.strip())}

  #print(vocabulary)

  final_tokens_per_tweet = []
  for data in preprocessed_data:
    final_tokens_per_tweet.append([token for token in data if token in vocabulary.keys()])

  #print(preprocessed_data)
  #print(final_tokens_per_tweet)

  word2id = dict()
  id2word = dict()
  vocabulary_size = len(vocabulary)
  count = 0
  for token in vocabulary.keys():
    word2id[token] = count
    id2word[count] = token
    count += 1

  #print(word2id)
  #print(id2word)

  directed_graph_adjacency_matrix = np.zeros((vocabulary_size, vocabulary_size))
  edge_weight_matrix = np.zeros((vocabulary_size, vocabulary_size))
  first_frequency = dict()
  last_frequency = dict()
  term_frequency = vocabulary
  strength = dict()
  degree = dict()
  selective_centraility = dict()


  for tweet in final_tokens_per_tweet:
    if len(tweet) == 0:
      continue
    if tweet[0] in first_frequency.keys():
      first_frequency[tweet[0]] += 1
    else:
      first_frequency[tweet[0]] = 1

    if tweet[-1] in last_frequency.keys():
      last_frequency[tweet[-1]] += 1
    else:
      last_frequency[tweet[-1]] = 1
    


    for i in range(len(tweet)-1):
      if tweet[i] == tweet[i+1]:
        continue
      x = word2id[tweet[i]]
      y = word2id[tweet[i+1]]
      directed_graph_adjacency_matrix[x][y] += 1

  for tweet in final_tokens_per_tweet:
    for i in range(len(tweet)-1):

      if tweet[i] == tweet[i+1]:
        continue
      x = word2id[tweet[i]]
      y = word2id[tweet[i+1]]

    # Updating degree..
      if tweet[i] in degree.keys():
        degree[tweet[i]] += 1
      else:
        degree[tweet[i]] = 1
        
      if tweet[i+1] in degree.keys():
        degree[tweet[i+1]] += 1
      else:
        degree[tweet[i+1]] = 1

      edge_weight_matrix[x][y] = directed_graph_adjacency_matrix[x][y]/(vocabulary[tweet[i]] + vocabulary[tweet[i+1]] - directed_graph_adjacency_matrix[x][y])

      if tweet[i] in strength.keys():
        strength[tweet[i]] += edge_weight_matrix[x][y]
      else:
        strength[tweet[i]] = edge_weight_matrix[x][y]




  first_frequency = {token:(first_frequency[token]/vocabulary[token] if token in first_frequency else 0) for token in vocabulary.keys()}
  last_frequency = {token:(last_frequency[token]/vocabulary[token] if token in last_frequency else 0) for token in vocabulary.keys()}
  degree = {token:(degree[token] if token in degree else 0) for token in vocabulary.keys()}
  strength = {token:(strength[token] if token in strength else 0) for token in vocabulary.keys()}
  selective_centraility = {token:(strength[token]/degree[token] if degree[token]!=0 else 0) for token in vocabulary.keys()}

  #print(degree)
  #print(vocabulary)

  maxdegree = max(degree.items(), key=lambda x: x[1])[1]
  max_degree_nodes_with_freq = {key:term_frequency[key] for key in degree.keys() if degree[key] == maxdegree}
  maxfreq = max(max_degree_nodes_with_freq.items(), key=lambda x: x[1])[1]
  central_node_name = [key for key in max_degree_nodes_with_freq.keys() if max_degree_nodes_with_freq[key] == maxfreq][0]
  #print("central node: ", central_node_name)

  # bfs
  distance_from_central_node = dict()
  central_node_id = word2id[central_node_name]
  q = [(central_node_id, 0)]

  # Set source as visited
  distance_from_central_node[central_node_name] = 0

  while q:
      vis = q[0]
      # Print current node
      #print(id2word[vis[0]], vis[1])
      q.pop(0)
        
      # For every adjacent vertex to
      # the current vertex
      for i in range(len(directed_graph_adjacency_matrix[vis[0]])):
          if (directed_graph_adjacency_matrix[vis[0]][i] == 1 and (id2word[i] not in distance_from_central_node.keys())):
              # Push the adjacent node
              # in the queue
              q.append((i, vis[1]+1))
              distance_from_central_node[id2word[i]] = vis[1]+1

  #print(distance_from_central_node)
  inverse_distance_from_central_node = {token:(1/distance_from_central_node[token] if token in distance_from_central_node and token != central_node_name else 0) for token in vocabulary.keys()}
  inverse_distance_from_central_node[central_node_name] = 1.0
  #print(inverse_distance_from_central_node)

  neighbour_importance = dict()

  for i in range(len(directed_graph_adjacency_matrix)):
    neighbours = set()

    # traversing outgoing edges
    for j in range(len(directed_graph_adjacency_matrix)):
      if i == j:
        continue
      if directed_graph_adjacency_matrix[i][j] > 0:
        neighbours.add(j)
    for j in range(len(directed_graph_adjacency_matrix)):
      if i == j:
        continue
      if directed_graph_adjacency_matrix[j][i] > 0:
          neighbours.add(j)
    if len(neighbours) != 0:
      neighbour_importance[id2word[i]] = sum([strength[id2word[j]] for j in neighbours])/len(neighbours)
    else:
      neighbour_importance[id2word[i]] = 0
      
  #print(neighbour_importance)

  unnormalized_node_weight = {node: (first_frequency[node] + last_frequency[node] + term_frequency[node] + selective_centraility[node] + inverse_distance_from_central_node[node] + neighbour_importance[node]) for node in vocabulary.keys()}
  max_node_weight = max(unnormalized_node_weight.items(), key=lambda x: x[1])[1]
  min_node_weight = min(unnormalized_node_weight.items(), key=lambda x: x[1])[1]
  #print("max node weight: ", max_node_weight, "min node weight: ", min_node_weight)
  normalized_node_weight = {node: ((unnormalized_node_weight[node] - min_node_weight)/(max_node_weight - min_node_weight) if max_node_weight != min_node_weight else unnormalized_node_weight[node]) for node in unnormalized_node_weight.keys()}
  #print("Unnormalized score: ", unnormalized_node_weight)
  #print("Normalized score: ", normalized_node_weight)

  damping_factor = 0.85
  relevance_of_node = {node: np.random.uniform(0,1,1)[0] for node in vocabulary.keys()}
  threshold = 0.000000001


  #print(relevance_of_node)

  count = 0
  while True:
    count += 1
    current_relevance_of_node = dict()
    for node in vocabulary.keys():
      outer_sum = 0
      node_idx = word2id[node]
      for j in range(len(directed_graph_adjacency_matrix)):
        if j == node_idx:
          continue
        if directed_graph_adjacency_matrix[j][node_idx] > 0:
          den_sum = 0
          for k in range(len(directed_graph_adjacency_matrix)):
            if k == j:
              continue
            den_sum += directed_graph_adjacency_matrix[j][k]
          outer_sum += ((directed_graph_adjacency_matrix[j][node_idx]/den_sum) * relevance_of_node[id2word[j]])
      current_relevance_of_node[node] = (1-damping_factor)*normalized_node_weight[node] + damping_factor*normalized_node_weight[node]*outer_sum
    

    # checking convergence..
    sq_error = sum([(current_relevance_of_node[node] - relevance_of_node[node])**2 for node in vocabulary.keys()])
    relevance_of_node = current_relevance_of_node
    if sq_error < threshold:
      break

  #print(relevance_of_node)
  #print(count)

  degree_centrality  = {node: 0 for node in vocabulary.keys()}

  if len(directed_graph_adjacency_matrix) > 1:
    for i in range(len(directed_graph_adjacency_matrix)):
      count = 0
      for j in range(len(directed_graph_adjacency_matrix)):
        if i == j:
          continue
        if directed_graph_adjacency_matrix[j][i] > 0:
          count += 1
      degree_centrality[id2word[i]] = count / (len(directed_graph_adjacency_matrix)-1)

  #print(degree_centrality)

  final_keyword_rank = [{'node': node, 'NE_rank': relevance_of_node[node], 'Degree': degree_centrality[node]} for node in vocabulary.keys()]

  #print("-----------")
  final_keyword_rank = sorted(final_keyword_rank, key = lambda i: (i['NE_rank'], i['Degree']), reverse = True)

  final_keywords = [keyword['node'] for keyword in final_keyword_rank]

  return final_keywords

In [ ]:
for tweet in df_query['Preprocessed_Data']:
  tweet_query.extend(tweet)

In [ ]:
tweet_query

['desperate',
 'let',
 'loose',
 'yatinarsinghanand',
 'violent',
 'rhetoric',
 'plottokillmodi',
 'hijab',
 'hate',
 'hateâ\x80¦',
 'yatinarsinghanand',
 'plottokillmodi',
 'hijab',
 'woman',
 'forced',
 'wear',
 'wear',
 'something',
 'hijab',
 'hijabisourpride',
 'hijabisfundamentalright',
 'hijaboruniform',
 'hijabcontroversy',
 'hijabcircular',
 'hijabisindividualright',
 'ð\x9f§\x95',
 'hijab',
 'hijabisourpride',
 'hijabisfundamentalright',
 'hijaboruniform',
 'hijabcontroversy',
 'hijabcircular',
 'hijabisindividualright',
 'protest',
 'u',
 'city',
 'karnataka',
 'hijab',
 'ban',
 'indian',
 'american',
 'ally',
 'demonstrated',
 'islamophobic',
 'unconstitutional',
 'ban',
 'school',
 'india',
 's',
 'nationalist',
 'bjp',
 'government',
 'take',
 'action',
 'canadaâ\x9e¡ï¸\x8f',
 'http',
 'tco957kbm8ajt',
 'take',
 'action',
 'globallyâ\x9e¡ï¸\x8f',
 'http',
 'tco5gy741dzp1',
 'http',
 'tcobvsjhjohkc',
 'karnataka',
 'hijab',
 'islamophobic',
 'bjp',
 'protest',
 'local',
 '

In [ ]:
keyword_dataset = df_query['tweet'].tolist()
tweet_query_keyword_extractor = keyword_extractor(keyword_dataset)

In [ ]:
print(keyword_dataset[0])
print(len(tweet_query))

They are so desperate they have let loose #YatiNarsinghanand again with his violent rhetoric. #PlotToKillModi to #hijab to hate and more hate.â¦
6398


In [ ]:
tweet_query_keyword_extractor

['hijab',
 'muslim',
 'women',
 'india',
 'wear',
 'muslims',
 'hijabrow',
 'islam',
 'allah',
 'like',
 'girls',
 'hijabisourright',
 'karnataka',
 'wearing',
 'amp',
 'school',
 'hijabisourpride',
 'ban',
 'you',
 'hijabisfundamentalright',
 'girl',
 'hijabcontroversy',
 'one',
 'education',
 'choice',
 'need',
 'world',
 'schools',
 'people',
 'islamic',
 'woman',
 'also',
 'want',
 'religion',
 'first',
 'bjp',
 'right',
 'even',
 'sitting',
 'hijabisindividualright',
 'leaders',
 'judge',
 'hijabban',
 'protest',
 'quietly',
 'students',
 'uniform',
 'tell',
 'allowed',
 'fighting',
 'colleges',
 'burqa',
 'issue',
 'started',
 'surely',
 'public',
 'decision',
 'religious',
 'said',
 'support',
 'indian',
 'rely',
 'alive',
 'abaya',
 'islamophobia',
 'freedom',
 'muslimah',
 'persecution',
 'allahuakbar',
 'sunnah',
 'see',
 'ramadan',
 'makkah',
 'alhamdulillah',
 'get',
 'islamicquotes',
 'dua',
 'quran',
 'rights',
 'hijabplot',
 'country',
 'hindu',
 'practice',
 'remove',
 

In [ ]:
tweet_keywords_yake = []
kw_extractor = yake.KeywordExtractor(top=20, stopwords=None)
keywords = kw_extractor.extract_keywords(' '.join(tweet_query))
#keywords = kw_extractor.extract_keywords(' '.join(df_query['tweet'].tolist()))
for kw, v in keywords:
  print("Keyphrase: ",kw, ": score", v)
  for key in kw.split():
    if(key.lower() not in tweet_keywords_yake):
      tweet_keywords_yake.append(key.lower())

print(tweet_keywords_yake)

Keyphrase:  hijab hijab hijab : score 3.538515818965514e-07
Keyphrase:  hijab karnataka india : score 4.4295050053984284e-07
Keyphrase:  muslim islam muslim : score 7.831784725969966e-07
Keyphrase:  hijab hijabisfundamentalright india : score 8.733034882458195e-07
Keyphrase:  hijabisourright hijabcontroversy hijab : score 1.2084471666090418e-06
Keyphrase:  karnataka india hijab : score 1.2402614015115601e-06
Keyphrase:  india hijab karnataka : score 1.2402614015115601e-06
Keyphrase:  muslim hijab started : score 1.3280528877404291e-06
Keyphrase:  hijab abaya muslim : score 1.3444833831980227e-06
Keyphrase:  hijab hijabrow http : score 1.425915298203952e-06
Keyphrase:  persecution muslim hijab : score 1.4602327688888964e-06
Keyphrase:  hijabcontroversy hijab hijabisfundamentalright : score 1.5153685192729542e-06
Keyphrase:  hijabisourright hijab http : score 1.6067046974268703e-06
Keyphrase:  wear hijab hijab : score 1.6311894877235963e-06
Keyphrase:  hijab muslim india : score 1.643581

In [ ]:
docs_preprocessed = []

In [ ]:
#Storing file name and data
total_documents = 0
path = '/content/drive/MyDrive/Tweelink_Dataset/Tweelink_Articles_Processed'
for filename in glob(os.path.join(path, '*')):
   with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
     filename = os.path.basename(f.name)
     data = json.load(f)
     d_date = data["Date"]
     if(d_date=="" or d_date=="Date"):
       continue
     format = '%Y-%m-%d'
 
     d_present_date = datetime.datetime.strptime(d_date, format)
 
     if(str(d_present_date.date()) not in [str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())]):
       continue
   
     docs_preprocessed.append({'Name':filename, 'Data':data})
     total_documents+=1
print(total_documents)

295


In [ ]:
print(docs_preprocessed[0])

{'Name': 'SuperBowl_10.json', 'Data': {'S. No.': '10', 'Base Hashtag': 'SuperBowl', 'Link': 'https://techcrunch.com/2022/02/19/this-week-in-apps-androids-privacy-sandbox-super-bowl-app-ads-app-annie-rebrands/', 'Date': '2022-02-20', 'Location': '', 'Title': 'This Week in Apps: Android’s Privacy Sandbox, Super Bowl app ads, App Annie rebrands', 'Body': 'Welcome back to This Week in Apps, the weekly TechCrunch series that recaps the latest in mobile OS news, mobile applications and the overall app economy. The app industry continues to grow, with a record number of downloads and consumer spending across both the iOS and Google Play stores combined in 2021, according to the latest year-end reports. App Annie says global spending across iOS, Google Play and third-party Android app stores in China grew 19% in 2021 to reach $170 billion. Downloads of apps also grew by 5%, reaching 230 billion in 2021, and mobile ad spend grew 23% year-over-year to reach $295 billion. In addition, consumers a

In [ ]:
def get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed):
  relevant_docs_list = []
  for doc in docs_preprocessed:
    if doc['Data']['Base Hashtag']==base_hashtag:
      current_date = datetime.datetime.strptime(base_date, format)
      prev_date = current_date - datetime.timedelta(days=1)
      next_date = current_date + datetime.timedelta(days=1)
      if(doc['Data']['Date'] in [str(prev_date.date()), str(current_date.date()), str(next_date.date())]):
        relevant_docs_list.append(doc['Name'])
  return relevant_docs_list

In [ ]:
def precision_at_k(k, base_hashtag, base_date, prediction_list, docs_preprocessed):
  relevant_docs_list = get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed)
  num_of_relevant_results=0
  for itr in range(k):
    if (prediction_list[itr][0] in relevant_docs_list):
      num_of_relevant_results+=1
  return num_of_relevant_results/k

In [ ]:
def mean_average_precision(max_k, base_hashtag, base_date, relevant_docs, docs_preprocessed):
  average_precision=0
  ctr=0
  for k_val in range(1,max_k+1):
    ctr+=1
    precision_at_k_val = precision_at_k(k_val, base_hashtag, base_date, relevant_docs, docs_preprocessed)
    #print('Hashtag: {}   Precision@{}: {}'.format(base_hashtag, k_val, precision_at_k_val))
    average_precision += precision_at_k_val
  return average_precision/ctr

In [ ]:
def recall_at_k(k, base_hashtag, base_date, prediction_list, docs_preprocessed):
  relevant_docs_list = get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed)
  current_num_of_relevant_results=0
  for itr in range(k):
    if (prediction_list[itr][0] in relevant_docs_list):
      current_num_of_relevant_results+=1
  if(len(relevant_docs_list)==0):
    return 0
  return current_num_of_relevant_results/len(relevant_docs_list)

In [ ]:
def mean_average_recall(max_k, base_hashtag, base_date, relevant_docs, docs_preprocessed):
  average_recall=0
  ctr=0
  for k_val in range(1,max_k+1):
    ctr+=1
    recall_at_k_val = recall_at_k(k_val, base_hashtag, base_date, relevant_docs, docs_preprocessed)
    #print('Hashtag: {}   Recall@{}: {}'.format(base_hashtag, k_val, recall_at_k_val))
    average_recall += recall_at_k_val
  return average_recall/ctr

In [ ]:
# def find_relevant_documents_cosine_similarity_count_vectorizer(docs_preprocessed, processed_query):
#   cosine_similarities_cv = {}
#   for document in docs_preprocessed:
#     query_sent = ' '.join(map(str, processed_query))
#     doc_text_sent = ' '.join(map(str, document['Data']['Body_processed']))
#     data = [query_sent, doc_text_sent]
#     count_vectorizer = CountVectorizer(encoding='latin-1', decode_error='ignore', ngram_range=(1,2))
#     vector_matrix = count_vectorizer.fit_transform(data)
#     cosine_similarity_matrix = cosine_similarity(vector_matrix)
#     cosine_similarities_cv[document['Name']] = cosine_similarity_matrix[0][1]
#   relevant_docs = list( sorted(cosine_similarities_cv.items(), key=operator.itemgetter(1),reverse=True))[:20]
#   for i in range(len(relevant_docs)):
#     for j in range(len(docs_preprocessed)):
#       if(relevant_docs[i][0] == docs_preprocessed[j]['Name']):
#         relevant_docs[i] = (relevant_docs[i][0], relevant_docs[i][1], docs_preprocessed[j]['Data']['Date'] )

#   return relevant_docs

In [ ]:
# def find_relevant_documents_cosine_similarity_tfidf_vectorizer(docs_preprocessed, processed_query):
#   cosine_similarities_tfidf = {}
#   for document in docs_preprocessed:
#     query_sent = ' '.join(map(str, processed_query))
#     doc_text_sent = ' '.join(map(str, document['Data']['Body_processed']))
#     data = [query_sent, doc_text_sent]
#     Tfidf_vect = TfidfVectorizer(encoding='latin-1', decode_error='ignore', ngram_range=(1,2))
#     vector_matrix = Tfidf_vect.fit_transform(data)
#     cosine_similarity_matrix = cosine_similarity(vector_matrix)
#     cosine_similarities_tfidf[document['Name']] = cosine_similarity_matrix[0][1]
#   relevant_docs = list( sorted(cosine_similarities_tfidf.items(), key=operator.itemgetter(1),reverse=True))[:20]
#   for i in range(len(relevant_docs)):
#     for j in range(len(docs_preprocessed)):
#       if(relevant_docs[i][0] == docs_preprocessed[j]['Name']):
#         relevant_docs[i] = (relevant_docs[i][0], relevant_docs[i][1], docs_preprocessed[j]['Data']['Date'] )

#   return relevant_docs

In [ ]:
print(tweet_query)
print(tweet_query_keyword_extractor)
print(tweet_keywords_yake)

['desperate', 'let', 'loose', 'yatinarsinghanand', 'violent', 'rhetoric', 'plottokillmodi', 'hijab', 'hate', 'hateâ\x80¦', 'yatinarsinghanand', 'plottokillmodi', 'hijab', 'woman', 'forced', 'wear', 'wear', 'something', 'hijab', 'hijabisourpride', 'hijabisfundamentalright', 'hijaboruniform', 'hijabcontroversy', 'hijabcircular', 'hijabisindividualright', 'ð\x9f§\x95', 'hijab', 'hijabisourpride', 'hijabisfundamentalright', 'hijaboruniform', 'hijabcontroversy', 'hijabcircular', 'hijabisindividualright', 'protest', 'u', 'city', 'karnataka', 'hijab', 'ban', 'indian', 'american', 'ally', 'demonstrated', 'islamophobic', 'unconstitutional', 'ban', 'school', 'india', 's', 'nationalist', 'bjp', 'government', 'take', 'action', 'canadaâ\x9e¡ï¸\x8f', 'http', 'tco957kbm8ajt', 'take', 'action', 'globallyâ\x9e¡ï¸\x8f', 'http', 'tco5gy741dzp1', 'http', 'tcobvsjhjohkc', 'karnataka', 'hijab', 'islamophobic', 'bjp', 'protest', 'local', 'dallas', 'muslim', 'jfk', 'square', 'downtown', 'draconian', 'hijab', 

#### Making own Corpus and applying Soft Cosine

In [ ]:
from gensim.similarities import SoftCosineSimilarity
from gensim.models import Word2Vec
from gensim import utils

# #Storing file name and data

# all_docs = []
# path = '/content/drive/MyDrive/Tweelink_Dataset/Tweelink_Articles_Processed'
# for filename in glob(os.path.join(path, '*')):
#    with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
#      filename = os.path.basename(f.name)
#      data = json.load(f)
#      d_date = data["Date"]
#      if(d_date=="" or d_date=="Date"):
#        continue
#      format = '%Y-%m-%d'
 
#      d_present_date = datetime.datetime.strptime(d_date, format)
 
#     #  if(str(d_present_date.date()) not in [str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())]):
#     #    continue
#      all_docs.append({'Name':filename, 'Data':data})

all_tweet_keywords = ' '.join(tweet_query)

whole_corpus_token_form = [article_data['Data']['Body_processed'] for article_data in docs_preprocessed]
whole_corpus_token_form.append(tweet_query_keyword_extractor)
print(len(whole_corpus_token_form))
# whole_corpus_token_form = [article_data['Data']['Body_processed'] for article_data in all_docs]
# whole_corpus_token_form.append(tweet_query_keyword_extractor)


def find_relevant_documents_soft_cosine_similarity(docs_preprocessed, processed_query):
  model = Word2Vec(sentences = whole_corpus_token_form, min_count=2) 
  termsim_index = WordEmbeddingSimilarityIndex(model.wv)
  sc_dictionary = Dictionary(whole_corpus_token_form)
  bow_corpus = [sc_dictionary.doc2bow(document) for document in whole_corpus_token_form]
  similarity_matrix = SparseTermSimilarityMatrix(termsim_index, sc_dictionary)
  docsim_index = SoftCosineSimilarity(bow_corpus, similarity_matrix, num_best=30)
  relevant_docs = docsim_index[sc_dictionary.doc2bow(processed_query)]
  iter = 0
  diff = 0
  for i in range(len(relevant_docs)):
    if relevant_docs[i][0] >= len(all_docs):
      continue
    relevant_docs[iter] = (all_docs[relevant_docs[i][0]]['Name'], relevant_docs[i][1], all_docs[relevant_docs[i][0]]['Data']['Date'])
    diff = i - iter
    iter += 1
  print(relevant_docs)
  return relevant_docs[:len(relevant_docs)-diff]

296


Plain Model (Soft Cosine Similarity)

In [ ]:
# Plain Model without YAKE / Keyword Extraction (Cosine Similarity Count Vectorizer)
relevant_docs_sc_plain = find_relevant_documents_soft_cosine_similarity(docs_preprocessed, tweet_query)

for rank, doc in enumerate(relevant_docs_sc_plain):
  print('Rank: {} Relevant Document: {}'.format(rank+1,doc))

print()

mean_average_precision_hashtag_sc_plain = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_sc_plain, docs_preprocessed)
print('Mean Average Precision Plain Model (Cosine Similarity Count Vectorizer) : {}'.format(mean_average_precision_hashtag_sc_plain))

mean_average_recall_hashtag_sc_plain = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_sc_plain, docs_preprocessed)
print('Mean Average Recall Plain Model (Cosine Similarity Count Vectorizer) : {}'.format(mean_average_recall_hashtag_sc_plain))

100%|██████████| 12563/12563 [00:22<00:00, 564.14it/s] 


[('SuperBowl_1.json', 1.0, '2022-02-14'), ('SuperBowl_2.json', 0.9578937292098999, '2022-02-15'), ('TaylorCatterall_213.json', 0.797200620174408, '2022-02-27'), ('TaylorCatterall_220.json', 0.7939330339431763, '2022-02-27'), ('Cyberpunk2077_60.json', 0.7851879596710205, '2022-02-16'), ('OttawaStrong_13.json', 0.7851879596710205, '2022-02-14'), ('TaylorCatterall_214.json', 0.7670072913169861, '2022-02-27'), ('sandhyamukherjee_65.json', 0.766011655330658, '2022-02-15'), ('DeepSidhu_79.json', 0.7607205510139465, '2022-02-28'), ('TaylorCatterall_219.json', 0.7564719915390015, '2022-02-27'), ('TaylorCatterall_215.json', 0.7488962411880493, '2022-02-27'), ('TaylorCatterall_216.json', 0.7473974227905273, '2022-02-27'), ('AK61_51.json', 0.7337390184402466, '2022-02-15'), ('TaylorCatterall_217.json', 0.7230721712112427, '2022-02-27'), ('DeepSidhu_80.json', 0.7102270126342773, '2022-02-21'), ('BattleOfBillingsBridge_32.json', 0.7099642753601074, '2022-02-14'), ('Wordle242_70.json', 0.70741617679

/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


In [ ]:
pip install annoy

In [ ]:
count = 0
for li in whole_corpus_token_form:
  if 'hijab' in li:
    print(li)
    count += 1
print(count)

['hijab', 'row', 'polaris', 'classroom', 'break', 'friendship', 'coastal', 'district', 'student', 'wearing', 'hijab', 'saffron', 'shawl', 'clash', 'campus', 'interfaith', 'friendship', 'classroom', 'rare', 'tenuous', '‘', 'normal', '’', 'circumstance', 'breaking', 'student', 'wearing', 'hijab', 'saffron', 'shawl', 'clash', 'campus', 'coastal', 'karnataka', 'coastal', 'district', 'situation', 'volatile', 'friend', 'classmate', 'turning', 'young', 'woman', 'targeted', 'wearing', 'hijab', 'udupi', 'district', 'recounted', 'felt', 'betrayed', 'classmate', 'friend', 'took', 'sporting', 'saffron', 'shawl', 'thairin', 'begum', 'student', 'government', 'junior', 'college', 'kundapura', 'said', 'shocked', 'see', 'friend', 'lead', 'saffron', 'shawl', 'protest', 'college', '“', 'gheraoed', 'u', '‘', 'jai', 'shri', 'ram', '’', 'slogan', 'spoken', 'since', '”', 'said', 'asmiya', 'student', 'rn', 'shetty', 'composite', 'pu', 'college', 'kundapura', 'said', 'one', 'close', 'friend', 'known', 'seven',

In [ ]:
from gensim.test.utils import common_texts as corpus, datapath
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex
from gensim.similarities.annoy import AnnoyIndexer

model = Word2Vec(whole_corpus_token_form, vector_size=50, min_count=5)  # train word-vectors
dictionary = Dictionary(whole_corpus_token_form)
tfidf = TfidfModel(dictionary=dictionary)
words = [word for word, count in dictionary.most_common(100)]
print(words)
word_vectors = model.wv.vectors_for_all(words, allow_inference=False)  # produce vectors for words in corpus
indexer = AnnoyIndexer(model.wv, num_trees=2)  # use Annoy for faster word similarity lookups
termsim_index = WordEmbeddingSimilarityIndex(model.wv, kwargs={'indexer': indexer})
similarity_matrix = SparseTermSimilarityMatrix(termsim_index, dictionary, tfidf)  # compute word similarities
tfidf_corpus = tfidf[[dictionary.doc2bow(document) for document in whole_corpus_token_form]]
docsim_index = SoftCosineSimilarity(tfidf_corpus, similarity_matrix, num_best=10)  # index tfidf_corpus
relevant_docs = docsim_index[dictionary.doc2bow(tweet_query)]
print(relevant_docs)
iter = 0
diff = 0
for i in range(len(relevant_docs)):
  if relevant_docs[i][0] >= len(all_docs):
    continue
  relevant_docs[iter] = (all_docs[relevant_docs[i][0]]['Name'], relevant_docs[i][1], all_docs[relevant_docs[i][0]]['Data']['Date'])
  diff = i - iter
  iter += 1
print(relevant_docs)


['system', 'user', 'trees', 'graph', 'computer', 'human', 'interface', 'response', 'survey', 'time', 'eps', 'minors']


100%|██████████| 12/12 [00:00<00:00, 1130.74it/s]

[(153, 1.0), (154, 1.0), (147, 1.0), (151, 1.0), (152, 1.0), (139, 1.0), (157, 1.0), (156, 1.0), (132, 1.0), (155, 1.0)]
[('PritiPatel_204.json', 1.0, '2022-02-26'), ('PritiPatel_205.json', 1.0, '2022-02-26'), ('IndiansInUkraine_198.json', 1.0, '2022-02-26'), ('PritiPatel_202.json', 1.0, '2022-02-26'), ('PritiPatel_203.json', 1.0, '2022-02-26'), ('Eurovision_190.json', 1.0, '2022-02-26'), ('PritiPatel_208.json', 1.0, '2022-02-26'), ('PritiPatel_207.json', 1.0, '2022-02-26'), ('Eurovision_183.json', 1.0, '2022-02-26'), ('PritiPatel_206.json', 1.0, '2022-02-26')]



/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


In [ ]:
for doc in docs_preprocessed:
  if doc['Name'] == 'MultiverseOfMadness_16.json':
    print(doc['Data'])

{'S. No.': '16', 'Base Hashtag': 'MultiverseOfMadness', 'Link': 'https://www.denofgeek.com/movies/doctor-strange-in-the-multiverse-of-madness-makes-what-if-essential-marvel-viewing/', 'Date': '2022-02-18', 'Location': '', 'Title': 'Doctor Strange in the Multiverse of Madness Makes What If Essential Marvel Viewing', 'Body': 'Marvel’s What If…? comic was always good for a nice, fun one-off story, but just because they were self-contained takes on alternate realities doesn’t mean they never mattered to Marvel Comics’ main continuity. An issue about the Avengers forming in the 1950s created the concept of the superhero team Agents of Atlas. Situations like the Spider-Man Clone Saga and Jane Foster as Thor first originated in that comic series. Even Skaar, Son of Hulk showed up in a What If…? story a week or so before appearing in regular continuity.\n\nThe animated What If…? show on Disney+ seemed like it was just a neat side project that meant nothing in the long run, but now that doesn’t

Model with Keyword Extractor (Soft Cosine Similarity)

In [ ]:
# Model with Keyword Extractor (Soft Cosine Similarity)
relevant_docs_sc_keyword_extractor = find_relevant_documents_soft_cosine_similarity(docs_preprocessed, tweet_query_keyword_extractor)

for rank, doc in enumerate(relevant_docs_sc_keyword_extractor):
  print('Rank: {} Relevant Document: {}'.format(rank+1,doc))

print()

mean_average_precision_hashtag_sc_keyword_extractor = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_sc_keyword_extractor, docs_preprocessed)
print('Mean Average Precision Plain Model (Cosine Similarity Count Vectorizer) : {}'.format(mean_average_precision_hashtag_sc_keyword_extractor))

mean_average_recall_hashtag_sc_keyword_extractor = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_sc_keyword_extractor, docs_preprocessed)
print('Mean Average Recall Plain Model (Cosine Similarity Count Vectorizer) : {}'.format(mean_average_recall_hashtag_sc_keyword_extractor))

100%|██████████| 12563/12563 [00:43<00:00, 286.29it/s]


[('SuperBowl_2.json', 1.0, '2022-02-15'), ('sandhyamukherjee_65.json', 1.0, '2022-02-15'), ('SuperBowl_1.json', 1.0, '2022-02-14'), ('DeepSidhu_75.json', 0.9331817626953125, '2022-02-17'), ('TaylorCatterall_220.json', 0.9147313833236694, '2022-02-27'), ('FullyFaltooNFTdrop_45.json', 0.9115850329399109, '2022-02-14'), ('SuperBowl_9.json', 0.9107582569122314, '2022-02-14'), ('DrDre_27.json', 0.9068968296051025, '2022-02-14'), ('TaylorCatterall_218.json', 0.8948929309844971, '2022-02-27'), ('BattleOfBillingsBridge_33.json', 0.894166111946106, '2022-02-14'), ('DeepSidhu_80.json', 0.8902300596237183, '2022-02-21'), ('DeepSidhu_79.json', 0.8893712162971497, '2022-02-28'), ('UNWFPtosavesoil_160.json', 0.8874863386154175, '2022-02-21'), ('Wordle242_70.json', 0.8868924379348755, '2022-02-11'), ('PritiPatel_204.json', 0.8864766955375671, '2022-02-26'), ('UNWFPtosavesoil_152.json', 0.8830447196960449, '2022-02-21'), ('SuperBowl_8.json', 0.8779916167259216, '2022-02-14'), ('sandhyamukherjee_64.jso

/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


Model with YAKE (Soft Cosine Similarity)

In [ ]:
# Model with YAKE (Soft Cosine Similarity)
relevant_docs_sc_yake = find_relevant_documents_soft_cosine_similarity(docs_preprocessed, tweet_keywords_yake)

for rank, doc in enumerate(relevant_docs_sc_yake):
  print('Rank: {} Relevant Document: {}'.format(rank+1,doc))

print()

mean_average_precision_hashtag_sc_yake = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_sc_yake, docs_preprocessed)
print('Mean Average Precision YAKE Model (Soft Cosine Similarity) : {}'.format(mean_average_precision_hashtag_sc_yake))

mean_average_recall_hashtag_sc_yake = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_sc_yake, docs_preprocessed)
print('Mean Average Recall YAKE Model (Soft Cosine Similarity) : {}'.format(mean_average_recall_hashtag_sc_yake))

100%|██████████| 12563/12563 [00:44<00:00, 279.54it/s]


[('IndiansInUkraine_199.json', 1.0, '2022-02-26'), ('IndiansInUkraine_195.json', 1.0, '2022-02-26'), ('IndiansInUkraine_192.json', 1.0, '2022-02-26'), ('IndiansInUkraine_191.json', 1.0, '2022-02-26'), ('qldfloods_180.json', 1.0, '2022-02-26'), ('SuperBowl_2.json', 1.0, '2022-02-15'), ('CNNPHVPDebate_170.json', 1.0, '2022-02-26'), ('CNNPHVPDebate_169.json', 1.0, '2022-02-26'), ('CNNPHVPDebate_168.json', 1.0, '2022-02-26'), ('CNNPHVPDebate_166.json', 1.0, '2022-02-26'), ('CNNPHVPDebate_164.json', 1.0, '2022-02-26'), ('CNNPHVPDebate_163.json', 1.0, '2022-02-26'), ('UNWFPtosavesoil_160.json', 1.0, '2022-02-21'), ('UNWFPtosavesoil_158.json', 1.0, '2022-02-21'), ('UNWFPtosavesoil_152.json', 1.0, '2022-02-21'), ('UNWFPtosavesoil_151.json', 1.0, '2022-02-21'), ('IStandWithTrudeau_98.json', 1.0, '2022-02-15'), ('IStandWithTrudeau_96.json', 1.0, '2022-02-17'), ('IStandWithTrudeau_95.json', 1.0, '2022-02-15'), ('IStandWithTrudeau_93.json', 1.0, '2022-02-15'), ('IStandWithTrudeau_92.json', 1.0, '2

/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: divide by zero encountered in true_divide
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)
/usr/local/lib/python3.7/dist-packages/gensim/similarities/termsim.py:382: RuntimeWarning: invalid value encountered in multiply
  normalized_corpus = np.multiply(corpus, 1.0 / corpus_norm)


**Across all hashtags**

In [ ]:
# global_list = [['narendramodi', '2022-02-14', 'india'],['UkraineRussiaCrisis', '2022-02-14', 'ukraine'],['IPL', '2022-02-14', 'india'],['TaylorSwift', '2022-02-14', 'USA'],['IndiaFightsCorona', '2022-02-14', 'india'],['narendramodi', '2022-02-15', 'india'],['UkraineRussiaCrisis', '2022-02-15', 'ukraine'],['IPL', '2022-02-15', 'india'],['TaylorSwift', '2022-02-15', 'USA'],['IndiaFightsCorona', '2022-02-15', 'india'],['narendramodi', '2022-02-16', 'india'],['UkraineRussiaCrisis', '2022-02-16', 'ukraine'],['IPL', '2022-02-16', 'india'],['TaylorSwift', '2022-02-16', 'USA'],['IndiaFightsCorona', '2022-02-16', 'india'],['narendramodi', '2022-02-17', 'india'],['UkraineRussiaCrisis', '2022-02-17', 'ukraine'],['IPL', '2022-02-17', 'india'],['TaylorSwift', '2022-02-17', 'USA'],['IndiaFightsCorona', '2022-02-17', 'india'],['narendramodi', '2022-02-18', 'india'],['UkraineRussiaCrisis', '2022-02-18', 'ukraine'],['IPL', '2022-02-18', 'india'],['TaylorSwift', '2022-02-18', 'USA'],['IndiaFightsCorona', '2022-02-18', 'india'],['narendramodi', '2022-02-19', 'india'],['UkraineRussiaCrisis', '2022-02-19', 'ukraine'],['IPL', '2022-02-19', 'india'],['hijab', '2022-02-19', 'india'],['vaccine', '2022-02-19', 'india'],['MillionAtIndiaPavilion', '2022-02-14', 'UAE'],['PunjabPanjeNaal', '2022-02-14', 'India'],['Euphoria', '2022-02-14', 'World'],['OscarsFanFavorite', '2022-02-14', 'World'],['ShameOnBirenSingh', '2022-02-14', 'india'],['BappiLahiri', '2022-02-16', 'india'],['BlandDoritos', '2022-02-16', 'USA'],['VERZUZ', '2022-02-16', 'USA'],['DragRaceUK', '2022-02-16', 'United Kingdom'],['BoycottWalgreens', '2022-02-18', 'USA'],['PunjabElections2022', '2022-02-20', 'india'],['WriddhimanSaha', '2022-02-20', 'india'],['stormfranklin', '2022-02-20', 'USA'],['QueenElizabeth', '2022-02-20', 'United Kingdom'],['ScottyFromWelding', '2022-02-20', 'Australia'],['CarabaoCupFinal', '2022-02-27', 'London'],['NZvSA', '2022-02-28', 'New Zealand'],['IPCC', '2022-02-28', 'Worldwide'],['SuperBowl', '2022-02-14', 'USA'],['MultiverseOfMadness', '2022-02-14', 'USA'],['Eminem', '2022-02-14', 'USA'],['IPLAuction', '2022-02-14', 'india'],['JohnsonOut21', '2022-02-14', 'United Kingdom'],['Cyberpunk2077', '2022-02-15', 'Worldwide'],['Wordle242', '2022-02-15', 'Worldwide'],['DeepSidhu', '2022-02-15', 'india'],['CanadaHasFallen', '2022-02-15', 'canada'],['IStandWithTrudeau', '2022-02-15', 'canada'],['CNNPHVPDebate', '2022-02-26', 'philippines'],['qldfloods', '2022-02-26', 'australia'],['Eurovision', '2022-02-26', 'worldwide'],['IndiansInUkraine', '2022-02-26', 'india'],['PritiPatel', '2022-02-26', 'united kingdom'],['TaylorCatterall', '2022-02-27', 'united kingdom'],['PSLFinal', '2022-02-27', 'pakistan'],['AustraliaDecides', '2022-02-27', 'australia'],['WorldNGODay', '2022-02-27', 'worldwide'],['TheBatman', '2022-02-28', 'USA'],['NationalScienceDay', '2022-02-28', 'india'],['msdtrong', '2022-02-14', 'india'],['Boycott_ChennaiSuperKings', '2022-02-14', 'india'],['GlanceJio', '2022-02-14', 'india'],['ArabicKuthu', '2022-02-14', 'india'],['Djokovic', '2022-02-15', 'australia'],['Real Madrid', '2022-02-15', 'santiago'],['bighit', '2022-02-15', 'korea'],['Maxwell', '2022-02-15', 'australia'],['mafsau', '2022-02-16', 'australia'],['channi', '2022-02-16', 'punjab'],['ayalaan', '2022-02-16', 'india'],['jkbose', '2022-02-16', 'india'],['HappyBirthdayPrinceSK', '2022-02-16', 'india'],['RandomActsOfKindnessDay', '2022-02-17', 'worldwide'],['happybirthdayjhope', '2022-02-17', 'korea'],['mohsinbaig', '2022-02-17', 'pakistan'],['aewdynamite', '2022-02-17', 'worldwide'],['aaraattu', '2022-02-17', 'india'],['ShivajiJayanti', '2022-02-18', 'india'],['PlotToKillModi', '2022-02-18', 'india'],['NationalDrinkWineDay', '2022-02-18', 'usa'],['HorizonForbiddenWest', '2022-02-18', 'worldwide'],['BoycottWalgreens', '2022-02-18', 'usa'],['CallTheMidwife', '2022-02-20', 'worldwide'],['OperationDudula', '2022-02-20', 'south africa'],['truthsocial', '2022-02-21', 'usa'],['nbaallstar', '2022-02-21', 'usa'],['shivamogga', '2022-02-21', 'india'],['HalftimeShow', '2022-02-14', 'usa'],['OttawaStrong', '2022-02-14', 'canada'],['DrDre', '2022-02-14', 'usa'],['BattleOfBillingsBridge', '2022-02-14', 'usa'],['FullyFaltooNFTdrop', '2022-02-14', 'worldwide'],['AK61', '2022-02-15', 'india'],['sandhyamukherjee', '2022-02-15', 'india'],['MUNBHA', '2022-02-15', 'worldwide'],['nursesstrike', '2022-02-15', 'australia'],['Realme9ProPlus', '2022-02-16', 'worldwide'],['KarnatakaHijabControversy', '2022-02-16', 'india'],['BJPwinningUP', '2022-02-16', 'india'],['Punjab_With_Modi', '2022-02-16', 'india'],['PushpaTheRule', '2022-02-16', 'india'],['RehmanMalik', '2022-02-22', 'india'],['harisrauf', '2022-02-22', 'pakistan'],['Rosettenville', '2022-02-22', 'south africa'],['NFU22', '2022-02-22', 'worldwide'],['justiceforharsha', '2022-02-22', 'india'],['wordle251', '2022-02-24', 'worldwide'],['ARSWOL', '2022-02-24', 'worldwide'],['stopwar', '2022-02-24', 'worldwide'],['PrayForPeace', '2022-02-24', 'worldwide'],['StopPutinNOW', '2022-02-24', 'worldwide'],['TeamGirlsCup', '2022-02-25', 'worldwide'],['Canucks', '2022-02-25', 'worldwide'],['PinkShirtDay', '2022-02-25', 'canada'],['superrugbypacific', '2022-02-25', 'australia']]

In [ ]:
# global_average_mean_average_precision_cs_cv = []
# global_mean_average_recall_cs_cv = []

# global_average_mean_average_precision_cs_tfidf = []
# global_mean_average_recall_cs_tfidf = []

# for iter in tqdm(range(len(global_list))):
#   u_base_hashtag = global_list[iter][0]
#   u_time = global_list[iter][1]
#   u_location = global_list[iter][2]
#   tweet_query = []
#   format = '%Y-%m-%d'
#   u_present_date = datetime.datetime.strptime(u_time, format)
#   u_prev_date = u_present_date - datetime.timedelta(days=1)
#   u_next_date = u_present_date + datetime.timedelta(days=1)
#   df_query = df.loc[df['hashtags'].str.contains(u_base_hashtag) & df['Date_Only'].isin([str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())])]

#   for tweet in df_query['Preprocessed_Data']:
#     tweet_query.extend(tweet)
  
#   # tweet_keywords = []
#   # kw_extractor = yake.KeywordExtractor(top=20, stopwords=None)
#   # keywords = kw_extractor.extract_keywords(' '.join(tweet_query))
#   # for kw, v in keywords:
#   #   #print("Keyphrase: ",kw, ": score", v)
#   #   for key in kw.split():
#   #     if(key not in tweet_keywords):
#   #       tweet_keywords.append(key)
  
#   docs_preprocessed = []

#   total_documents = 0
#   path = '/content/drive/MyDrive/Tweelink_Dataset/Tweelink_Articles_Processed'
#   for filename in glob(os.path.join(path, '*')):
#     with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
#       filename = os.path.basename(f.name)
#       data = json.load(f)
#       d_date = data["Date"]
#       if(d_date=="" or d_date=="Date"):
#         continue
#       format = '%Y-%m-%d'
  
#       d_present_date = datetime.datetime.strptime(d_date, format)
  
#       if(str(d_present_date.date()) not in [str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())]):
#         continue
    
#       docs_preprocessed.append({'Name':filename, 'Data':data})
#       total_documents+=1
  
#   relevant_docs_cs_cv = find_relevant_documents_cosine_similarity_count_vectorizer(docs_preprocessed, tweet_query)
#   mean_average_precision_hashtag_cs_cv = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_cs_cv, docs_preprocessed)
#   global_average_mean_average_precision_cs_cv.append(mean_average_precision_hashtag_cs_cv)
#   mean_average_recall_hashtag_cs_cv = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_cs_cv, docs_preprocessed)
#   global_mean_average_recall_cs_cv.append(mean_average_recall_hashtag_cs_cv)

#   relevant_docs_cs_tfidf = find_relevant_documents_cosine_similarity_tfidf_vectorizer(docs_preprocessed, tweet_query)
#   mean_average_precision_hashtag_cs_tfidf = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_cs_tfidf, docs_preprocessed)
#   global_average_mean_average_precision_cs_tfidf.append(mean_average_precision_hashtag_cs_tfidf)
#   mean_average_recall_hashtag_cs_tfidf = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_cs_tfidf, docs_preprocessed)
#   global_mean_average_recall_cs_tfidf.append(mean_average_recall_hashtag_cs_tfidf)

In [ ]:
# # cs cv
# overall_average_mean_average_precision_cs_cv = sum(global_average_mean_average_precision_cs_cv)/len(global_average_mean_average_precision_cs_cv)
# print(overall_average_mean_average_precision_cs_cv)

# overall_mean_average_recall_cs_cv = sum(global_mean_average_recall_cs_cv)/len(global_mean_average_recall_cs_cv)
# print(overall_mean_average_recall_cs_cv)

In [ ]:
# # cs tfidf
# overall_average_mean_average_precision_cs_tfidf = sum(global_average_mean_average_precision_cs_tfidf)/len(global_average_mean_average_precision_cs_tfidf)
# print(overall_average_mean_average_precision_cs_tfidf)

# overall_mean_average_recall_cs_tfidf = sum(global_mean_average_recall_cs_tfidf)/len(global_mean_average_recall_cs_tfidf)
# print(overall_mean_average_recall_cs_tfidf)